In [1]:
import pandas as pd

In [32]:
data_name = "HelpSteer"

if data_name in ['flask', "HelpSteer"]:
    df = pd.read_json(f"data/spearman/{data_name}_preprocessed.json", lines=True)
    df.rename(columns={"human_score":"score"}, inplace=True)
elif data_name=="fb_bench":
    df = pd.read_parquet("data/1228/Feedback-Bench/data/train-00000-of-00001-eddf1add30d20be1.parquet")
    df.rename(columns={"orig_score":"score"}, inplace=True)

In [33]:
template_feedback = '''### Task Description:
An instruction (might include an Input inside it), a response to evaluate and a score rubric representing a evaluation criteria are given.
- Write a detailed feedback that assess the quality of the response strictly based on the given score rubric, not evaluating in general.
- After writing a feedback, write a score that is an integer between 1 and 5. You should refer to the score rubric.

### The instruction to evaluate:
{instruction}

### Response to evaluate:
{output}

### Score Rubrics:
[{Definition}]
- Score 1: {criteria_for_1}
- Score 2: {criteria_for_2}
- Score 3: {criteria_for_3}
- Score 4: {criteria_for_4}
- Score 5: {criteria_for_5}

### Output format:
The output format should look as follows: \"Feedback: <write your feedback here> Score: <give your score here>\"
Remember that you should end with the the score. Please do not generate any other opening, closing, and explainations.'''

template_no_feedback = '''### Task Description:
An instruction (might include an Input inside it), a response to evaluate and a score rubric representing a evaluation criteria are given.
- Provide a single score that is an integer between 1 and 5 to assess the quality of the response strictly based on the given score rubric, not evaluating in general.

### The instruction to evaluate:
{instruction}

### Response to evaluate:
{output}

### Score Rubrics:
[{Definition}]
- Score 1: {criteria_for_1}
- Score 2: {criteria_for_2}
- Score 3: {criteria_for_3}
- Score 4: {criteria_for_4}
- Score 5: {criteria_for_5}

### Output format:
The output format should look as follows: \"score: <the score you give>\".
Remember that you only need to provide the score. There is no need to generate explaination for it.'''

In [34]:
def fill_prompt(row, data_name, feedback=False):
    template = template_feedback if feedback else template_no_feedback
    if data_name in ['flask', "HelpSteer"]:
        if data_name=="flask":
            evaluation_aspect = row['criteria']
            evaluation_criteria = row['criteria_dict'][evaluation_aspect]
        elif data_name=="HelpSteer":
            evaluation_aspect = row['criteria']
            evaluation_criteria = eval(row['criteria_dict'])[evaluation_aspect]
        res = template.format(
            Definition=evaluation_criteria["definition"],
            criteria_for_1=evaluation_criteria["scoring_standards"]["1"],
            criteria_for_2=evaluation_criteria["scoring_standards"]["2"],
            criteria_for_3=evaluation_criteria["scoring_standards"]["3"],
            criteria_for_4=evaluation_criteria["scoring_standards"]["4"],
            criteria_for_5=evaluation_criteria["scoring_standards"]["5"],
            instruction=row['instruction'], 
            output=row['response']
        )
        return res
    
    elif data_name=="fb_bench":
        res = template.format(
            Definition=row["orig_criteria"],
            criteria_for_1=row["orig_score1_description"],
            criteria_for_2=row["orig_score2_description"],
            criteria_for_3=row["orig_score3_description"],
            criteria_for_4=row["orig_score4_description"],
            criteria_for_5=row["orig_score5_description"],
            instruction=row['orig_instruction'], 
            output=row['orig_response']
        )
        return res

df['user_prompt'] = df.apply(lambda s: fill_prompt(s, data_name, feedback=False), axis=1)
df['user_prompt_feedback'] = df.apply(lambda s: fill_prompt(s, data_name, feedback=False), axis=1)

In [35]:
df = df[['user_prompt', "user_prompt_feedback", "score"]]
df.to_json(f"data/1228/{data_name}.json", orient="records", lines=True)

In [36]:
df

,user_prompt,user_prompt_feedback,score
0,### Task Description:\nAn instruction (might i...,### Task Description:\nAn instruction (might i...,3
1,### Task Description:\nAn instruction (might i...,### Task Description:\nAn instruction (might i...,5
2,### Task Description:\nAn instruction (might i...,### Task Description:\nAn instruction (might i...,2
3,### Task Description:\nAn instruction (might i...,### Task Description:\nAn instruction (might i...,2
4,### Task Description:\nAn instruction (might i...,### Task Description:\nAn instruction (might i...,3
...,...,...,...
8940,### Task Description:\nAn instruction (might i...,### Task Description:\nAn instruction (might i...,2
8941,### Task Description:\nAn instruction (might i...,### Task Description:\nAn instruction (might i...,4
8942,### Task Description:\nAn instruction (might i...,### Task Description:\nAn instruction (might i...,4
8943,### Task Description:\nAn instruction (might i...,### Task Description:\nAn instruction (might i...,2
